In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset
from datasets import IterableDataset
from tqdm import tqdm
import json
import random
import os

2025-05-20 08:48:34.912892: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747730915.146521      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747730915.213247      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# LLM as judge generation assessment

In this notebook some randomly chosen MCQ questions that were generated by LLM are assessed for compliance with the prompt that was used for the generation to instruct model.

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()


In [3]:

# Set your HF token & username as environment variables
os.environ["HF_TOKEN"] = user_secrets.get_secret("HF_TOKEN")
# Replace with your username)
os.environ["HF_USERNAME"] = "dimvsometimes"

## Judge model
Google's Gemma-3 is used as one of strong models

In [4]:
torch.random.manual_seed(0)
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-4b-it",
    device_map="cuda",
    #device_map='cpu',
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it", padding_side='left')

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [5]:
text = lambda msg: tokenizer.apply_chat_template(
    msg,
    tokenize=False,
    add_generation_prompt=True
)



In [6]:
generation_args = {
    "max_new_tokens": 250,
    "return_full_text": False,
    "temperature": 0.2,
    "do_sample": True,
}

## The question set taken for prompt alignment assessment
Each pair of term and context that were randomly chosen got 6 generated questions:
- for each type of Bloom's taxonomy of questions considered: Remember type (the lowest one), Comprehension type and Application type
 - MCQ were generated with 0-shot prompt and one/few-shot prompt

In [7]:
import pandas as pd
import ast
test_data = pd.read_csv("/kaggle/input/mcq-validation/prompt_validation_data.csv", index_col=None).iloc[:, 1:]
test_data

,Term,Context,Remember_0shot,Remember_1shot,Comprehension_0shot,Comprehension_fewshot,Application_0shot,Application_1shot
0,translation,Another common use of machine translation is t...,"{\n 'Question': 'What does CAT stand for?',...","{\n 'Question': ""In what process do machine...","{\n 'Question': 'What does CAT stand for?',...","{\n 'Question': ""What does CAT stand for?"",...","{\n 'Question': 'What does CAT stand for?',...","{\n 'Question': ""What does CAT stand for?"",..."
1,question answering,Let`s begin by introducing the bidirectional t...,{\n 'Question': 'In what way does the left-...,"{\n 'Question': ""In what type of task is it...",{\n 'Question': 'What is an example of a ta...,"{\n 'Question': ""Explain why the left-to-ri...",{\n 'Question': 'What type of model does no...,"{\n 'Question': ""What is an example of a ta..."
2,coherence relation,Coherence relations arose from the independent...,{\n 'Question': 'In what way did Hobbs cont...,"{\n 'Question': ""Which scholar first introd...",{\n 'Question': 'What did Asher and Lascari...,"{\n 'Question': ""What are some methods othe...",{\n 'Question': 'Which scholar first introd...,"{\n 'Question': ""Which approach does not di..."
3,question answering,Some of the question datasets described above ...,{\n 'Question': 'In what type of QA tasks d...,"{\n 'Question': ""In what type of dataset do...","{\n 'Question': 'What does the term ""open b...","{\n 'Question': ""What distinguishes the ope...",{\n 'Question': 'What type of QA tasks invo...,"{\n 'Question': ""What type of QA task does ..."
4,dialogue system,It is a common practice for dialogue systems t...,{\n 'Question': 'What is another function o...,"{\n 'Question': ""What is another function o...",{\n 'Question': 'What two main functions do...,"{\n 'Question': ""What two functions does th...",{\n 'Question': 'What is another function o...,"{\n 'Question': ""What is another function o..."
...,...,...,...,...,...,...,...,...
95,relation extraction,The great advantage of unsupervised relation e...,{\n 'Question': 'What is one significant be...,"{\n 'Question': ""What is an example of a re...",{\n 'Question': 'What is one of the main ad...,"{\n 'Question': ""Explain how unsupervised r...",{\n 'Question': 'Which statement best descr...,"{\n 'Question': ""What challenge does unsupe..."
96,lexicon,The Computational Grammar Coder (CGC) of Klein...,{\n 'Question': 'What did the CGC use for i...,Term: computational grammar coder.\n\nContext:...,{\n 'Question': 'What did Klein and Simmons...,"{\n 'Question': ""What was the primary diffe...",{\n 'Question': 'What did the CGC use for i...,"{\n 'Question': ""What is the primary compon..."
97,dialogue,The goal of text-to-speech (TTS) systems is to...,{\n 'Question': 'What are TTS systems prima...,"{\n 'Question': ""In what application are TT...",{\n 'Question': 'What are TTS systems prima...,"{\n 'Question': ""What are some potential ap...",{\n 'Question': 'What are TTS systems prima...,"{\n 'Question': ""What is the first step in ..."
98,discourse coherence,Training models to predict longer contexts tha...,{\n 'Question': 'What does training models ...,"{\n 'Question': ""What type of task does the...",{\n 'Question': 'What does training models ...,"{\n 'Question': ""Explain how training model...",{\n 'Question': 'How does training models t...,"{\n 'Question': ""What is an effective strat..."


In [13]:
batch_size = 5
pipe = pipeline(
    "text-generation",
  
    model=model,
    tokenizer=tokenizer,
    batch_size=batch_size
)

Device set to use cuda


## The judge prompt for verdicts

In [10]:
message_expl = lambda q: [
    {"role": "system", "content": """
    For the provided list of prompt instructions, determine whether each instruction has been followed in the LLM actual output.
    Please generate a list of JSON with two keys: `verdict` and `reason`.
    The 'verdict' key should STRICTLY be either a 'yes' or 'no'. Only answer 'yes' if the instruction COMPLETELY follows the instruction, and 'no' otherwise.
    You should be EXTRA STRICT AND CAREFUL when giving a 'yes'.
    The 'reason' is the reason for the verdict.
    Provide a 'reason' ONLY if the answer is 'no'.
    The provided prompt instructions are the instructions to be followed in the prompt, which you have no access to.
    IMPORTANT: Please make sure to only return in JSON format, with the 'verdicts' key mapping to a list of JSON objects.
    """
     },
    {"role": "user", "content": """
    Example input: For MT beam widths k give us k hypotheses at the end. We can pass all k to the downstream application with their respective scores, or if we just need a single translation we can pass the most probable hypothesis.
Example term: 'Hydrology'.
Example answer:  Beam width k determines the number of hypotheses generated by the model, which affects the diversity of the output.
Example actual output: In machine translation (MT), what is typically used for beam width k when generating translations?
Example prompt instructions: ["Question in LLM Actual Output is based on the information available only from Input.", "Answer is completely wrong to the question in actual output accoring to Input", "LLM Actual Output does not completely contain, spoil or provide Answer explicitly.", "Question in actual output is related to the term."]
Example JSON:
{
    "verdicts": [
        {
            "verdict": "yes"
        },
        {
            "verdict": "no",
            "reason": "The answer corresponds to Input and answers the question in the LLM output overall."
        },
        {
            "verdict": "yes",
        },
        {
            "verdict": "no",
            "reason": "The LLM output is not related to the term 'Hydrology' and is related to another topic."
        }
    ]  
}
The number of 'verdicts' MUST BE STRICTLY EQUAL to the number of prompt instructions.

    """+q}
]

## Criteria for assessment

Below the criteria that judge will make its verdicts based on are:
- Context correspondence
- Term correspondence
- Answer correctness
- Bloom correspondence (assessed separately a bit further)

The criteria are based on what's passed in MCQ generation prompt to agent so it's expected that generated questions follow the required prompt instructions

In [14]:
response_cols = ['Remember_0shot', 'Remember_1shot',
       'Comprehension_0shot', 'Comprehension_fewshot', 'Application_0shot',
       'Application_1shot']

metrics = ['Context correspondence', 'Term correspondence', 'Answer correctness']
positive_response=["yes", "yes", "no"]


instructions=['Question in LLM Actual Output is based on the information available only from Input.', 
              "Question in LLM Actual Output is related to the term according to Input.", 
            'LLM Actual Output does not completely contain, spoil or provide content of Answer explicitly.']

metrics_by_prompt = {}

In [15]:
for col in response_cols:
    
    cnt_not_dict = test_data[~test_data[col].str.contains('{\n')].shape[0]
    print(f'{col}: {cnt_not_dict}') # only one time LLM failed to generate dictionary

Remember_0shot: 0
Remember_1shot: 1
Comprehension_0shot: 0
Comprehension_fewshot: 0
Application_0shot: 0
Application_1shot: 0


In [53]:
test_data

for col in [response_cols[5]]:
    metrics_res = {m: 0 for m in metrics}

    subdataset = test_data[test_data[col].str.contains('{\n')]
    
    def gen():
        for i, row in subdataset.iterrows():
            
            
            if '{\n' not in row[col]:
                continue
            if row[col].count('{') != row[col].count('}'):
                continue
            start_i = row[col].find('{')
            end_i = row[col].rfind('}')
      
            llm_dict = row[col][start_i:end_i+1].replace("'User's dialogue acts'", "'Users dialogue acts'")
            llm_answer = ast.literal_eval(llm_dict)
            question = llm_answer['Question']
            answer = llm_answer['Options'][llm_answer['Correct answer'][:2]].replace("User's dialogue acts", "User\'s dialogue acts")
            
            yield {"Message": text(message_expl(
                        f"""Prompt Instructions:
                    [{', '.join(f'"{instr}"' for instr in instructions)}]
                    
                    Input:
                    {row['Context']}
                    
                    Term:
                    '{row['Term']}'
                    
                    Answer: 
                    {answer}
                    
                    LLM Actual Output:
                    {question}
                    
                    JSON output:
                    """))}

    
    dataset = Dataset.from_generator(gen)
    #j=0
    corr_cnt = 0
    
    for j, out in tqdm(enumerate(pipe(KeyDataset(dataset, 'Message'), **generation_args)),
        total=test_data.shape[0]):
        response = out[0]['generated_text']
        response_dict = ast.literal_eval(response[8:-3])['verdicts']

        for verd, metric, pos_resp in zip(response_dict, metrics, positive_response):
            if verd['verdict']==pos_resp:
                metrics_res[metric]+=1
        corr_cnt+=1
    metrics_res['total'] = corr_cnt
    metrics_by_prompt[col] = metrics_res

Generating train split: 0 examples [00:00, ? examples/s]

100%|██████████| 100/100 [19:19<00:00, 11.60s/it]


In [54]:
metrics_by_prompt

{'Remember_0shot': {'Context correspondence': 95,
  'Term correspondence': 97,
  'Answer correctness': 95,
  'total': 100},
 'Remember_1shot': {'Context correspondence': 93,
  'Term correspondence': 95,
  'Answer correctness': 98,
  'total': 98},
 'Comprehension_0shot': {'Context correspondence': 98,
  'Term correspondence': 99,
  'Answer correctness': 96,
  'total': 100},
 'Comprehension_fewshot': {'Context correspondence': 98,
  'Term correspondence': 100,
  'Answer correctness': 97,
  'total': 100},
 'Application_0shot': {'Context correspondence': 95,
  'Term correspondence': 96,
  'Answer correctness': 96,
  'total': 100},
 'Application_1shot': {'Context correspondence': 96,
  'Term correspondence': 97,
  'Answer correctness': 97,
  'total': 100}}

In [18]:
df_metrics = pd.DataFrame(columns=['Prompt']+metrics)



In [55]:
key = 'Application_1shot'
i=5
df_metrics.loc[i, :] = [key]+[round(metrics_by_prompt[key][m]/metrics_by_prompt[key]['total']*100, 2) for m in metrics]

In [56]:
df_metrics

,Prompt,Context correspondence,Term correspondence,Answer correctness
0,Remember_0shot,95.0,97.0,95.0
1,Remember_1shot,94.9,96.94,100.0
2,Comprehension_0shot,98.0,99.0,96.0
3,Comprehension_fewshot,98.0,100.0,97.0
4,Application_0shot,95.0,96.0,96.0
5,Application_1shot,96.0,97.0,97.0


In [57]:
df_metrics.to_csv('prompt_judge_res.csv')

Bloom correspondence is measured based on the indicator-words (specific words that are related to each type of questions in the taxonomy) that were used in the generation prompt.

In [58]:
bloom_instr = {'Remember': 'Question in LLM Actual Output includes question words like "What","Why" or "How".',
                'Comprehension': 'Question in LLM Actual Output includes a word like "explain", "summarize", "rephrase", "compare" or their synonyms',
               'Application': 'Question in LLM Actual Output includes a verb like "develop", "organize", "build", "plan" or their synonyms'
                ''}

In [76]:
#bloom_taxonomy_correspondence
bloom_metric='Bloom correspondence'

for col in [response_cols[5]]:
    metrics_res = {m: 0 for m in [bloom_metric]}
    bloom_instr_str = [bloom_instr[col.split('_')[0]]]

    subdataset = test_data[test_data[col].str.contains('{\n')]
    
    def gen():
        for i, row in subdataset.iterrows():
            
            if '{\n' not in row[col]:
                continue
            if row[col].count('{') != row[col].count('}'):
                continue
            start_i = row[col].find('{')
            end_i = row[col].rfind('}')

            llm_dict = row[col][start_i:end_i+1].replace("'User's dialogue acts'", "'Users dialogue acts'")
            llm_answer = ast.literal_eval(llm_dict)
            question = llm_answer['Question']
            answer = llm_answer['Options'][llm_answer['Correct answer'][:2]].replace("User's dialogue acts", "User\'s dialogue acts")
           
            yield {"Message": text(message_expl(
                        f"""Prompt Instructions:
                    [{', '.join(f'"{instr}"' for instr in instructions)}]
                    
                    Input:
                    {row['Context']}
                    
                    Term:
                    '{row['Term']}'
                    
                    Answer: 
                    {answer}
                    
                    LLM Actual Output:
                    {question}
                    
                    JSON output:
                    """))}

    
    dataset = Dataset.from_generator(gen)
 
    corr_cnt = 0
    
    for j, out in tqdm(enumerate(pipe(KeyDataset(dataset, 'Message'), **generation_args)),
        total=test_data.shape[0]):
        response = out[0]['generated_text']
        response_dict = ast.literal_eval(response[8:-3])['verdicts']

        for verd, metric, pos_resp in zip(response_dict, [bloom_metric], ['yes']):
            if verd['verdict']==pos_resp:
                metrics_res[metric]+=1
        corr_cnt+=1
    metrics_res['total'] = corr_cnt
    metrics_by_prompt[col][bloom_metric] = metrics_res[bloom_metric]

100%|██████████| 100/100 [19:27<00:00, 11.68s/it]


In [77]:
metrics_by_prompt

{'Remember_0shot': {'Context correspondence': 95,
  'Term correspondence': 97,
  'Answer correctness': 95,
  'total': 100,
  'Bloom correspondence': 95},
 'Remember_1shot': {'Context correspondence': 93,
  'Term correspondence': 95,
  'Answer correctness': 98,
  'total': 98,
  'Bloom correspondence': 93},
 'Comprehension_0shot': {'Context correspondence': 98,
  'Term correspondence': 99,
  'Answer correctness': 96,
  'total': 100,
  'Bloom correspondence': 98},
 'Comprehension_fewshot': {'Context correspondence': 98,
  'Term correspondence': 100,
  'Answer correctness': 97,
  'total': 100,
  'Bloom correspondence': 98},
 'Application_0shot': {'Context correspondence': 95,
  'Term correspondence': 96,
  'Answer correctness': 96,
  'total': 100,
  'Bloom correspondence': 95},
 'Application_1shot': {'Context correspondence': 96,
  'Term correspondence': 97,
  'Answer correctness': 97,
  'total': 100,
  'Bloom correspondence': 96}}

In [78]:
df_metrics[bloom_metric] = [round(metrics_by_prompt[col][bloom_metric]/metrics_by_prompt[col]['total']*100, 2) for col in response_cols]
df_metrics

,Prompt,Context correspondence,Term correspondence,Answer correctness,Bloom correspondence
0,Remember_0shot,95.0,97.0,95.0,95.0
1,Remember_1shot,94.9,96.94,100.0,94.9
2,Comprehension_0shot,98.0,99.0,96.0,98.0
3,Comprehension_fewshot,98.0,100.0,97.0,98.0
4,Application_0shot,95.0,96.0,96.0,95.0
5,Application_1shot,96.0,97.0,97.0,96.0


In [79]:
df_metrics.to_csv('prompt_judge_res.csv')

# Evaluation prompt reference:
Adjusted prompt of **deepeval** framework for prompt alignment assessment

In [4]:
!pip install deepeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 10.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.9/168.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [109]:
from deepeval.models.base_model import DeepEvalBaseLLM

class JudgeLLM(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model
        self.gen_pars = {
            "max_new_tokens": 250,
            "return_full_text": False,
            "temperature": 0.01,
            "do_sample": True,
        }

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
        generated_ids = model.generate(**model_inputs, max_new_tokens=100, temperature=0.01)
        tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        gen_text = tokenizer.batch_decode(generated_ids[:, model_inputs['input_ids'][0].shape[0]:])[0]
        print(prompt, gen_text)
        return gen_text

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
        generated_ids = model.generate(**model_inputs, max_new_tokens=100, temperature=0.01)
        tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        gen_text = tokenizer.batch_decode(generated_ids[:, model_inputs['input_ids'][0].shape[0]:])[0]
        print(prompt, gen_text)
        return gen_text

    def get_model_name(self):
        return "Qwen2.5"

In [ ]:
judge_model = JudgeLLM(model=model)

In [99]:
from deepeval import evaluate
from deepeval.metrics import HallucinationMetric
from deepeval.test_case import LLMTestCase


context=["Alignment Knowing the minimum edit distance is useful for algorithms like finding potential spelling error corrections. But the edit distance algorithm is important in another way; with a small change, it can also provide the minimum cost alignment between two strings. Aligning two strings is useful throughout speech and language processing. In speech recognition, minimum edit distance alignment is used to compute the word error rate (Chapter 16). Alignment plays a role in machine translation, in which sentences in a parallel corpus (a corpus with a text in two languages) need to be matched to each other. "]


In [100]:
from deepeval import evaluate
from deepeval.metrics import PromptAlignmentMetric
from deepeval.test_case import LLMTestCase

metric = PromptAlignmentMetric(
    async_mode=False,
    prompt_instructions=["""Using the information in the context and given term generate question
    related to the term according to context. 
    in question's text and starts with "What","Why" or "How"""],
    model=judge_model,
    include_reason=True,
)
test_case = LLMTestCase(
    input="Alignment Knowing the minimum edit distance is useful for algorithms like finding potential spelling error corrections. But the edit distance algorithm is important in another way; with a small change, it can also provide the minimum cost alignment between two strings. Aligning two strings is useful throughout speech and language processing. In speech recognition, minimum edit distance alignment is used to compute the word error rate (Chapter 16). Alignment plays a role in machine translation, in which sentences in a parallel corpus (a corpus with a text in two languages) need to be matched to each other. ",

    actual_output="In speech recognition, what is the purpose of using minimum edit distance alignment?"
)

In [111]:
evaluate(test_cases=[test_case], metrics=[metric])

✨ You're running DeepEval's latest Prompt Alignment Metric! (using Qwen2.5, strict=False, async_mode=True)...

Evaluating 1 test case(s) in parallel: |          |  0% (0/1) [Time Taken: 00:02, ?test case/s]

For the provided list of prompt instructions, determine whether each instruction has been followed in the LLM actual output.
Please generate a list of JSON with two keys: `verdict` and `reason`.
The 'verdict' key should STRICTLY be either a 'yes' or 'no'. Only answer 'yes' if the instruction COMPLETELY follows the instruction, and 'no' otherwise.
You should be EXTRA STRICT AND CAREFUL when giving a 'yes'.
The 'reason' is the reason for the verdict.
Provide a 'reason' ONLY if the answer is 'no'. 
The provided prompt instructions are the instructions to be followed in the prompt, which you have no access to.

**
IMPORTANT: Please make sure to only return in JSON format, with the 'verdicts' key mapping to a list of JSON objects.
Example input: What number is the stars of the sky?
Example actual output: HEY THERE! I think what you meant is "What is the number of stars in the sky", but unfortunately I don't know the answer to it.
Example prompt instructions: ["Answer the input in a well-man

KeyError: 'verdicts'

##### 